In [1]:
from pyspark.sql import SparkSession
import pyspark.sql.functions as F
import pyspark.sql.types as T
from pyspark.sql.window import Window as W

spark = SparkSession.builder.appName('Puzzle').getOrCreate()

In [2]:
df = spark.read.csv('data/puzzle_015.csv', header=True, inferSchema=True)
df.show()

+-----------+--------+-----------+
|WorkOrderID|STEP_NBR|STEP_STATUS|
+-----------+--------+-----------+
|      AA100|       0|          C|
|      AA100|       1|          W|
|      AA100|       2|          W|
|      AA200|       0|          W|
|      AA200|       1|          W|
|      AA300|       0|          C|
|      AA300|       1|          C|
+-----------+--------+-----------+



In [14]:
df.withColumn(
            'Rn', F.when((F.col('STEP_NBR')==0) & (F.col('STEP_STATUS') =='C'), 1)
                    .when(F.col('STEP_STATUS')=='W', 1)
                    .otherwise(0)
).groupBy('WorkOrderID').agg(
    F.sum(F.col("Rn")).alias('Rn'), 
    F.count(F.col('STEP_STATUS').isin(['W'])).alias('TotalCount'),
    F.count_distinct(F.col('STEP_STATUS')).alias('Dcnt')
).where(
    (F.col('Rn')==F.col('TotalCount')) & 
    (F.col('Dcnt') ==2)
).show()

+-----------+---+----------+----+
|WorkOrderID| Rn|TotalCount|Dcnt|
+-----------+---+----------+----+
|      AA100|  3|         3|   2|
+-----------+---+----------+----+

